# Load libraries

In [100]:
# pip install fsspec / conda install -c conda-forge fsspec
# pip install s3fs
import pandas as pd
import numpy as np
import boto3
import os
from sagemaker import get_execution_role

role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


# Load data from one file

In [29]:
bucket_name = 'eliezerraj-908671954593-dataset'
prefix_name = 'payment/glue-job'
file_name = 'run-1713493427814-part-r-00000'

data_location = 's3://{}/{}/{}'.format(bucket_name, prefix_name, file_name)

df = pd.read_csv(data_location)

print("---------------------------------")
print(f"Shape of dataframe '{df.shape}':")

---------------------------------
Shape of dataframe '(2589, 25)':


In [30]:
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
df.head()

,id,fk_card_id,card_number,terminal_name,coord_x,coord_y,card_type,card_model,payment_at,mcc,amount,night_day,ic_night_day,wkend_wkday,ic_wkend_wkday,day_of_year,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_tx,time_btw_cc_tx,fraud
0,26965,843,111.111.000.843,TERM-221,222.0,261.0,CREDIT,VIRTUAL,2024-03-19 21:03:38.0,GAS,840.0,night,1,wkday,0,79.0,4,857.25,6,659.17,14,408.07,63,26,1
1,37036,636,111.111.000.636,TERM-59,38.0,34.0,CREDIT,VIRTUAL,2024-04-18 18:57:19.0,LAUNDRY,142.0,day,0,wkday,0,109.0,1,142.00,3,274.67,10,254.30,149,0,0
2,28164,538,111.111.000.538,TERM-24,77.0,45.0,CREDIT,CHIP,2024-03-23 15:47:16.0,SPORTING,408.0,day,0,wkend,1,83.0,3,209.33,3,209.33,6,207.17,8,1802,0
3,41128,517,111.111.000.517,TERM-97,8.0,85.0,CREDIT,CHIP,2024-04-30 18:52:27.0,CINEMA,106.0,day,0,wkday,0,121.0,2,109.50,2,109.50,8,210.75,22,997,0
4,37698,56,111.111.000.056,TERM-48,94.0,54.0,CREDIT,CHIP,2024-04-20 17:46:19.0,LAUNDRY,106.0,day,0,wkend,1,111.0,2,303.00,7,551.00,13,743.23,64,0,0


# Load data from a chunk of files in a folder

In [101]:
s3 = boto3.client('s3')

bucket_name = 'eliezerraj-908671954593-dataset'
prefix_name = 'payment/glue-job/'

# List objects in the specified bucket and prefix
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix_name)

# Extract the list of file keys from the response
file_keys = [obj['Key'] for obj in response.get('Contents', [])]

In [102]:
# Read each file
df = pd.DataFrame()

for obj_key in file_keys[1:]:
    # Read the object content
    obj_content = s3.get_object(Bucket=bucket_name, Key=obj_key)['Body'].read()
    print(f"Content of file '{obj_key}':")

    data_location = 's3://{}/{}'.format(bucket_name, obj_key)

    df_temp = pd.read_csv(data_location)
    df = pd.concat([df, df_temp], ignore_index=True)
    print(f"Shape of chunk '{df.shape}':")
    print("---------------------------------")

print("---------------------------------")
print(f"Shape of dataframe '{df.shape}':")


Content of file 'payment/glue-job/run-1713493427814-part-r-00000':
Shape of chunk '(2589, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00001':
Shape of chunk '(5179, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00002':
Shape of chunk '(7769, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00003':
Shape of chunk '(10359, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00004':
Shape of chunk '(12949, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00005':
Shape of chunk '(15539, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00006':
Shape of chunk '(18129, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00007':
Shape of chunk '(20

In [103]:
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
df.head(3)

,id,fk_card_id,card_number,terminal_name,coord_x,coord_y,card_type,card_model,payment_at,mcc,amount,night_day,ic_night_day,wkend_wkday,ic_wkend_wkday,day_of_year,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_tx,time_btw_cc_tx,fraud
0,26965,843,111.111.000.843,TERM-221,222.0,261.0,CREDIT,VIRTUAL,2024-03-19 21:03:38.0,GAS,840.0,night,1,wkday,0,79.0,4,857.25,6,659.17,14,408.07,63,26,1
1,37036,636,111.111.000.636,TERM-59,38.0,34.0,CREDIT,VIRTUAL,2024-04-18 18:57:19.0,LAUNDRY,142.0,day,0,wkday,0,109.0,1,142.00,3,274.67,10,254.30,149,0,0
2,28164,538,111.111.000.538,TERM-24,77.0,45.0,CREDIT,CHIP,2024-03-23 15:47:16.0,SPORTING,408.0,day,0,wkend,1,83.0,3,209.33,3,209.33,6,207.17,8,1802,0


In [104]:
df.to_csv('payment.csv', index=False, header=True)

In [110]:
# Upload
bucket_name = 'eliezerraj-908671954593-dataset'
prefix_name_load_file = 'payment/notebook'

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix_name_load_file, 'payment.csv')).upload_file('payment.csv')

# Featuring Engeniring

In [111]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role

from sagemaker import Session
import sagemaker
from sagemaker import get_execution_role
from sagemaker.workflow.parameters import ( 
 ParameterInteger,
 ParameterString,
 ParameterFloat)

In [112]:
processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)

In [113]:
framework_version = "1.0-1"
sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    role=role,
    instance_type="ml.m5.large",
    instance_count=1,
    base_job_name='mlops-payment-feature-eng'
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


In [114]:
bucket_name = 'eliezerraj-908671954593-dataset'
prefix_name = 'payment/notebook'
file_name = 'payment.csv'

In [115]:
input_source = 's3://{}/{}/{}'.format(bucket_name, prefix_name, file_name)
data_location = 's3://{}/{}/{}'.format(bucket_name, prefix_name, 'requirements.txt')
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix_name_load_file, 'requirements.txt')).upload_file('requirements.txt')

print("---------------------------------")
print(f"bucket_name '{bucket_name}':")
print(f"prefix_name '{prefix_name}':")
print(f"input_source '{input_source}':")
print(f"data_location '{data_location}':")

---------------------------------
bucket_name 'eliezerraj-908671954593-dataset':
prefix_name 'payment/notebook':
input_source 's3://eliezerraj-908671954593-dataset/payment/notebook/payment.csv':
data_location 's3://eliezerraj-908671954593-dataset/payment/notebook/requirements.txt':


In [116]:
processor_args = sklearn_processor.run(
    inputs=[
        ProcessingInput(source=input_source,
                        destination="/opt/ml/processing/input"),
        ProcessingInput(source=data_location,
                        destination="/opt/ml/processing/input/req/")
    ],
    outputs=[
        ProcessingOutput(output_name="train",
                         source="/opt/ml/processing/train",
                         destination=f"s3://{bucket_name}/{prefix_name}/output/train"),
        ProcessingOutput(output_name="validation",
                         source="/opt/ml/processing/validation",
                         destination=f"s3://{bucket_name}/{prefix_name}/output/validation"),
        ProcessingOutput(output_name="test",
                         source="/opt/ml/processing/test",
                         destination=f"s3://{bucket_name}/{prefix_name}/output/test")],
    code=f"feat-v02.py",
)

INFO:sagemaker:Creating processing-job with name mlops-payment-feature-eng-2024-04-22-00-14-28-959


............................Collecting imbalanced-learn
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 7.4 MB/s eta 0:00:00
/opt/ml/processing/input/code/feat-v02.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["distance"][ind] = distance
datetime64[ns]
train_data:  (36256, 13)
validation_data:  (10359, 13)
test_data:  (5180, 13)
Fraudulent transactions are 22.14% of the train_data.
Fraudulent transactions are 21.99% of the test_data.
Fraudulent transactions are 11.00% of the validation_data.
Fraudulent transactions AFTER SMOTE are 50.00% of the train_data.
## Processing completed. Exiting.

